# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json
import gmaps
# import json
import os

# Import API key
from config import key

In [2]:
# Load city data from WeatherPy
city_data = "../WeatherPy/Output/city_data.csv"
city_data_df = pd.read_csv(city_data, index_col=[0], encoding="ISO-8859-1")
city_data_df.head()
city_data_df.dtypes

City_Name       object
City_Lat       float64
City_Long      float64
Temperature    float64
Humidity         int64
Cloudiness       int64
Wind_Speed     float64
Country         object
Date            object
dtype: object

In [3]:
# Rename columns and create new dataframe with dropped NA values 
city_data_df = city_data_df.rename(columns={'City_Name': 'City', 'City_Lat': 'Latitude', 
                                            'City_Long': 'Longitude', 'Wind_Speed': 'Wind Speed'})

city_loc = city_data_df.dropna(axis = 0, how ='any')
# city_data_cleandf = city_data_dropna[(city_data_dropna != 0).all(1)]

# comparing sizes of data frames
print("Old data frame length:", len(city_data_df), "\nNew data frame length:", 
       len(city_loc), "\nNumber of rows with at least 1 NA value: ",
       (len(city_data_df)-len(city_loc)))
city_loc.to_csv("./output_data/city_loc.csv")

Old data frame length: 645 
New data frame length: 639 
Number of rows with at least 1 NA value:  6


### Humidity Heatmap

In [4]:
# Configure gmaps
gmaps.configure(api_key=key)

# Set Locations for the heatmap
locations = city_loc[["Latitude", "Longitude"]]

humidity = city_loc["Humidity"].astype(float)

In [6]:
# Plot humidity heatmap
fig = gmaps.figure()

# Create humidity heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)
# Add layer and display fig
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Ideal Weather Conditions

In [7]:
# Create new dataframe based on indicated parameters
ideal_cities = city_loc[(city_loc['Temperature']>70) & (city_loc['Temperature']<90) 
                   & (city_loc['Humidity']>30) & (city_loc['Humidity']<70) 
                   & (city_loc['Wind Speed']<10) & (city_loc['Cloudiness']<30)]
ideal_cities.head()
# len(ideal_cities)    #---- used to check number of cities that meet criteria; uncomment to enable 

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
13,Rome,34.2570,-85.1647,84.45,42,1,1.99,US,2021-06-17 17:57:41
14,Guarapari,-20.6667,-40.4975,73.89,68,14,9.42,BR,2021-06-17 17:59:43
19,Saint Andrews,34.0454,-81.1293,84.56,38,1,3.00,US,2021-06-17 17:59:45
32,Jablah,35.3614,35.9256,71.89,68,0,4.61,SY,2021-06-17 17:59:49
62,Chantilly,38.8943,-77.4311,75.87,35,0,3.00,US,2021-06-17 18:00:00


### Find Nearest Hotel

In [8]:
# Create new dataframe of city geo data; add new column to store hotel name
hotels_df = ideal_cities.loc[:, ['City', 'Country', 'Latitude', 'Longitude', 'Date']].copy()
hotels_df.head()

,City,Country,Latitude,Longitude,Date
13,Rome,US,34.2570,-85.1647,2021-06-17 17:57:41
14,Guarapari,BR,-20.6667,-40.4975,2021-06-17 17:59:43
19,Saint Andrews,US,34.0454,-81.1293,2021-06-17 17:59:45
32,Jablah,SY,35.3614,35.9256,2021-06-17 17:59:49
62,Chantilly,US,38.8943,-77.4311,2021-06-17 18:00:00


In [9]:
# Add column to hotels dataframe
hotels_df['Hotel Name'] = ""

# Parameters dictionary for Nearby search requests
params = {
#     'location': 'city_geo',
    'types': 'lodging', # keyword
    'radius': 5000, 
    'key': key
}
# Nearby search base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through pandas dataframe
for i, row in hotels_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params['location'] = f'{lat},{long}'


    try:

        # Construct url and make API request
        response = requests.get(base_url, params)  #=params
        json_results = response.json()
        hotel_name = json_results['results'][0]['name']
        hotels_df['Hotel Name'][i] = hotel_name      
#         hotels.append(json_results)['results'][0]['name']
    except:
        print('Missing field for city of {}...skipping'.format(row['City']))

        print('-'*26)
hotels_df.to_csv('../VacationPy/hotels_df.csv')
hotels_df.head(20)

C:\Users\jchan\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Missing field for city of Avera...skipping
--------------------------
Missing field for city of Albany...skipping
--------------------------


,City,Country,Latitude,Longitude,Date,Hotel Name
13,Rome,US,34.2570,-85.1647,2021-06-17 17:57:41,La Quinta Inn & Suites by Wyndham Rome
14,Guarapari,BR,-20.6667,-40.4975,2021-06-17 17:59:43,Hotel Porto do Sol
19,Saint Andrews,US,34.0454,-81.1293,2021-06-17 17:59:45,Baymont by Wyndham Columbia Northwest
32,Jablah,SY,35.3614,35.9256,2021-06-17 17:59:49,مدرسة سعد بن ابي وقاص الابتدائيه
62,Chantilly,US,38.8943,-77.4311,2021-06-17 18:00:00,Hampton Inn Washington-Dulles Int'l Airport South
73,Broome,US,42.2506,-75.8330,2021-06-17 18:00:10,Chenango Valley State Park
79,Hamilton,US,39.1834,-84.5333,2021-06-17 18:00:10,Six Acres Bed & Breakfast
86,Statesboro,US,32.4488,-81.7832,2021-06-17 18:00:16,Holiday Inn Statesboro-University Area
99,Russell,US,32.3502,-85.1999,2021-06-17 18:00:06,Macs
154,Avera,US,33.1940,-82.5271,2021-06-17 17:58:47,
